# VNAxKEPCO Data Analysis Tutorial

Welcome to the VNAxKEPCO Data Analysis Tutorial! This guide will walk you through the process of analyzing data from your VNAxKEPCO experiments using the provided Python script. The tutorial is divided into sections to cover different aspects of the analysis.

## Prerequisites

Before you start, make sure you have the required software and libraries installed:

- Python: Install Python from [python.org](https://www.python.org/downloads/).
- Jupyter Notebook: Install Jupyter Notebook by running `pip install jupyter` in your terminal.
- Required Libraries: Install the necessary Python libraries using the following command:

```bash
pip install numpy matplotlib pandas scipy pyvisa
```

## Section 1: 1 Port Experiment

### 1.1 Initializing 1 Port Experiment

```python
from VNAxKEPCO_1P import Experiment1P

E1 = Experiment1P()
```

### 1.2 Frequency Sweep

This part customizes experiment parameters such as frequency range, magnetic fields, and sample name. Then, it runs the experiment using E1.MeasureRef (to measure the reference) and E1.Measure methods.

```python
ref_field = 400
start_freq = 0.5
stop_freq = 6
step = 20e6  # 20 MHz
start_freq, stop_freq = start_freq * 1e9, stop_freq * 1e9
num_points = int((stop_freq - start_freq) / step) + 1
fields = np.linspace(0, 400, 201)

# Change this
sample = 'NiFe'
file_name = f'1_Port_{sample}_{start_freq/1e9}-{stop_freq/1e9}_GHz_@_{fields.min()}-{fields.max()}_Oe'
dir = fr'C:\Users\physlab\Desktop\VNA Automation\Data\{file_name}'

# Run the experiment
E1.MeasureRef(ref_field, start_freq, stop_freq, num_points, dir)
E1.Measure(start_freq, stop_freq, num_points, fields, dir)
```

### 1.3 Data Loading

The Load.NPZ_1P function is used to load data from the saved NPZ file, returning information, frequency, magnetic field, reference S11, and S11 data.

```python
dir = r'c:\Users\physlab\Desktop\VNA Automation\Data\{filename}'

Info, freq, field, RefS11, S11 = Load.NPZ_1P(dir)
```

### 1.4 S11 Plots

Visualizing S11 plots at a specified frequency using the Plot.S_parameter function.

```python
specified_freq = 3
Plot.S_parameter(freq, field, specified_freq, S11)
```

### 1.5 Set Frequency and Field Bounds

Setting frequency and field bounds for further analysis using the Crop.cut_freq and Crop.cut_field functions.

```python
lower_freq = 0.5
upper_freq = 4
lower_field = 0
upper_field = 200

freq_used = Crop.cut_freq(freq, lower_freq, upper_freq)[0]
lower_freq = Crop.cut_freq(freq, lower_freq, upper_freq)[1]
upper_freq = Crop.cut_freq(freq, lower_freq, upper_freq)[2]

field_used = Crop.cut_field(field, lower_field, upper_field)[0]
lower_field = Crop.cut_freq(field, lower_field, upper_field)[1]
upper_field = Crop.cut_freq(field, lower_field, upper_field)[2]
```

### 1.6 Visualizing Filtered Data

Applying filters (Filter_Spline, FilterBG_Median, FilterBG_MedianH) to the raw data and visualizing both raw and filtered data.

```python
abspow = Calc.AbsPower1P(S11, RefS11)  # Power Absorbed

# Apply filters
filteredData = fil.Filter_Spline(abspow, a=0.1)
filteredData = fil.FilterBG_Median(filteredData)
filteredData = fil.FilterBG_MedianH(filteredData)

# Crop data
cutData = filteredData[lower_freq:upper_freq, lower_field:upper_field]

# Plotting
plt.figure(figsize=(8,3))

plt.subplot(1,2,1)
plt.pcolormesh(field, freq, abspow, cmap='coolwarm')
plt.colorbar()
plt.ylabel('Frequency (GHz)')
plt.xlabel('Field (Oe)')
plt.title('Raw Plot')

plt.subplot(1,2,2)
plt.pcolormesh(field_used, freq_used, cutData, cmap='coolwarm')  #gist_heat #inferno #plasma #jet #gist_grey #coolwarm 
plt.colorbar()
plt.xlabel('Field (Oe)')
plt.ylabel('Frequency (GHz)')
plt.title('Filtered Plot')

plt.subplots_adjust(left=0.2, right=0.9, bottom=1, top=1.5, wspace=0.6, hspace=1)
```

### 1.7 Single dP/dH Plot

Generating a single dP/dH plot for a specific frequency using the E1.Measure_dPdH function and saving the results to a CSV file.

```python
frequency = 7
field = field_used

d = Plot.Fdata(cutData, F=frequency, h=field_used, freq=freq_used)
max_field = field[np.argmax(d)]
field_below, field_above = 150, 150
lower = max_field - field_below
if lower < 0:
    lower = 0

upper = max_field + field_above
fields = np.linspace(lower, upper, round((field_below + field_above) / 2) + 1)
frequency *= 1e9
data = E1.Measure_dPdH(frequency, fields, file_name=None, read_reps=3, delta_H=5)
df = pd.DataFrame({'Fields': fields, 'Data': data})

# Convert DataFrame to CSV (Set Directory)
sample = 'NiFe'
file_name = f'1_Port_dPdH_{sample}_{frequency/1e9}_GHz_@_{fields.min()}-{fields.max()}_Oe'
dir = fr'C:\Users\plyslab\Desktop\VNA Automation\Data\{file_name}.csv'
df.to_csv(dir, index=False)
```

### 1.8 Shift dP/dH (only considering positive fields)

Looping through multiple frequencies, shifting dP/dH, and saving the results to CSV files.

```python
start = 1
stop = 5
interval = 0.5
frequencies = np.arange(start, stop+interval, interval)
field = field_used

for fr in frequencies:
    d = Plot.Fdata(cutData, F=fr, freq=freq_used)
    max_field = field[np.argmax(d)]
    field_below, field_above = 150, 150
    lower = max_field - field_below
    if lower < 0:
        lower = 0

    upper = max_field + field_above
    fields = np.linspace(lower, upper, round((field_below + field_above) / 2) + 1)
    fr *= 1e9
    data = E1.Measure_dPdH(fr, fields, file_name=None, read_reps=3, delta_H=5, returnData=True)
    df = pd.DataFrame({'Fields': fields, 'Data': data})

    # Convert DataFrame to CSV (Set Directory)
    sample = 'NiFe'
    file_name = f'1_Port_dPdH_{sample}_{fr/1e9}_GHz_@_{fields.min()}-{fields.max()}_Oe'
    dir = fr'C:\Users\plyslab\Desktop\VNA Automation\Data\{file_name}.csv'
    df.to_csv(dir, index=False)
```

## Section 2: 2 Port Experiment

### 2.1 Initializing 2 Port Experiment

```python
from VNAxKEPCO_2P import Experiment2P

E2 = Experiment2P()
```

### 2.2 Frequency Sweep

Customizing experiment parameters and running the 2-port experiment.

```python
ref_field = 500
start_freq = 0.1
stop_freq = 5
step = 20e6  # 20MHz
start_freq, stop_freq = start_freq * 1e9, stop_freq * 1e9
num_points = int((stop_freq - start_freq) / step) + 1

fields = np.linspace(-350, 350, 301)

sample = 'NiFe'
file_name = f'2Port_{sample}_{round(start_freq/1e9, 1)}-{round(stop_freq/1e9, 1)}_GHz_@_{fields.min()}-{fields.max()}_Oe'
dir = fr'C:\Users\physlab\Desktop\VNA Automation\Data\2port\{file_name}'

# Run the experiment
E2.MeasureRef(ref_field, start_freq, stop_freq, num_points, dir)
E2.Measure(start_freq, stop_freq, num_points, fields, dir)
```

### 2.3 Data Loading

Using Load.NPZ_2P to load data from the saved NPZ file.

```python
dir = r'c:\Users\plyslab\Desktop\VNA Automation\Data\2port\{filename}'

Info, freq, field, RefS11, RefS21, RefS22, RefS12, S11, S12, S21, S22 = Load.NPZ_2P(dir)
```

### 2.4 S-Parameter Plots (S11 & S21)

Visualizing S11 and S21 plots at a specified frequency using Plot.S_parameter.

```python
specified_freq = 2
Plot.S_parameter(freq, field, S11, S21)
```

### 2.5 Set Frequency and Field Bounds

Setting frequency and field bounds for further analysis.

```python
lower_freq = 0.5
upper_freq = 5
lower_field = -350
upper_field = 350

freq_used = Crop.cut_freq(freq, lower_freq, upper_freq)[0]
lower_freq = Crop.cut_freq(freq, lower_freq, upper_freq)[1]
upper_freq = Crop.cut_freq(freq, lower_freq, upper_freq)[2]

field_used = Crop.cut_field(field, lower_field, upper_field)[0]
lower_field = Crop.cut_freq(field, lower_field, upper_field)[1]
upper_field = Crop.cut_freq(field, lower_field, upper_field)[2]
```

### 2.6 Visualize Raw & Filtered Plots

Applying filters and visualizing both raw and filtered data for the 2-port experiment.

```python
abspow = Calc.AbsPower2P(S11, S21, RefS11, RefS21)

# Apply filters
filteredData = fil.Filter_Spline(abspow, a=0.1)
filteredData = fil.FilterBG_Median(filteredData)
filteredData = fil.FilterBG_MedianH(filteredData)

# Crop data
cutData = filteredData[lower_freq:upper_freq, lower_field:upper_field]

# Density Plots
plt.figure(figsize=(8,3))
cmap = 'coolwarm'

plt.subplot(1,2,1)
plt.pcolormesh(field, freq, abspow, cmap=cmap)
plt.colorbar()
plt.ylabel('Frequency (GHz)')
plt.xlabel('Field (Oe)')
plt.title('Raw Plot')

plt.subplot(1,2,2)
plt.pcolormesh(field_used, freq_used, cutData, cmap=cmap)
plt.colorbar()
plt.grid(False)
plt.xlabel('Field (Oe)')
plt.ylabel('Frequency (GHz)')
plt.title('Filtered Plot')

plt.subplots_adjust(left=0.2, right=0.9, bottom=1, top=1.5, wspace=0.6, hspace=1)

```

### 2.7 Line Plots For Specific Field and Frequency

Generating a single dP/dH plot for the 2-port experiment, similar to 1.7.

```python
# Field
plt.figure(figsize=(6, 3))
Plot.plotH(cutData, H=100, f=freq_used, field=field_used)
# ...

# Frequency
plt.figure(figsize=(6, 3))
Plot.plotF(cutData, F=2, h=field_used, freq=freq_used)
# ...
```

### 2.8 Single dP/dH Plot

Generating a single dP/dH plot for the 2-port experiment, similar to 1.7.

```python
frequency = 2 * 1e9
field_below = 80
field_above = 80
lower = Calc.concentrated_dpdh(cutData, frequency, field_used, freq_used*1e9, field_below, field_above)[0]
upper = Calc.concentrated_dpdh(cutData, frequency, field_used, freq_used*1e9, field_below, field_above)[1]
fields = np.linspace(lower, upper, round((field_below + field_above) / 2) + 1)

data = E2.Measure_dPdH(frequency, fields, file_name=None, read_reps=3, delta_H=5, returnData=True)
df = pd.DataFrame({'Fields': fields, 'dPdH Data': data})

# Convert DataFrame to CSV (Set Directory)
sample = 'NiFe'
file_name = f'2_Port_dPdH_{sample}_{frequency/1e9}_GHz_@_{fields.min()}-{fields.max()}_Oe'
dir = fr'C:\Users\plyslab\Desktop\VNA Automation\Data\2port\{file_name}.csv'
df.to_csv(dir, index=False)
```

### 2.9 Shift dP/dH (only considering positive fields)

Looping through multiple frequencies, shifting dP/dH, saving results to CSV, and visualizing the line plots of measured dP/dH data.

```python
start = 2
stop = 5
interval = 0.1
frequencies = np.arange(start, stop + interval, interval)
field = field_used

for fr in frequencies:
    d = Plot.Fdata(cutData, F=fr, freq=freq_used)
    max_field = field[np.argmax(d)]
    field_below, field_above = 100, 100
    lower = max_field - field_below
    if lower < 0:
        lower = 0

    upper = max_field + field_above
    fields = np.linspace(lower, upper, round((field_below + field_above) / 2) + 1)
    fr *= 1e9
    data = E2.Measure_dPdH(fr, fields, file_name=None, read_reps=3, delta_H=5, returnData=True)
    df = pd.DataFrame({'Fields': fields, 'dPdH Data': data})

    # Convert DataFrame to CSV (Set Directory)
    sample = 'NiFe'
    file_name = f'2_Port_dPdH_{sample}_{round(fr/1e9, 1)}_GHz_@_{fields.min()}-{fields.max()}_Oe'
    dir = fr'C:\Users\plyslab\Desktop\VNA Automation\Data\2port\{file_name}.csv'
    df.to_csv(dir, index=False)

    # Visualize the line plots of the measured dP/dH data
    directory_path = r'C:\Users\plyslab\Desktop\VNA Automation\Data\2port'
    Plot.shift_dPdH(directory_path)
```

This tutorial covers the key steps for acquiring and analyzing both 1 Port and 2 Port experiment data. Feel free to explore and modify the provided script based on your specific experimental setup and requirements.


-----------------------

**Developer:** Mahad Naveed

**Supervisor:** Dr. Sabieh Anwar

**Mentor:** Dr. Adnan Raza